PART 1

In [20]:
import pandas as p

'''
    1. mpg:           continuous
    2. cylinders:     multi-valued discrete
    3. displacement:  continuous
    4. horsepower:    continuous
    5. weight:        continuous
    6. acceleration:  continuous
    7. model year:    multi-valued discrete
    8. origin:        multi-valued discrete
    9. car name:      string (unique for each instance)
'''

def discretizer(frame, continousVariables):
    for var in continousVariables:
        frame[var] = p.cut(data[var], bins= [0, data[var].median(), data[var].max()], labels= ['low', 'high'])
    return frame

conVars = ['mpg', 'displacement', 'horsepower', 'weight', 'acceleration']

data = p.read_csv('auto-mpg.csv')

data = discretizer(data, conVars)
data.astype('category')
data = data.drop('car name', axis=1)
dataStr = data.astype(str)
data

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,low,8,high,high,high,low,70,1
1,low,8,high,high,high,low,70,1
2,low,8,high,high,high,low,70,1
3,low,8,high,high,high,low,70,1
4,low,8,high,high,high,low,70,1
...,...,...,...,...,...,...,...,...
94,high,4,low,low,low,low,74,2
95,high,4,low,low,low,high,74,3
96,high,4,low,low,low,high,74,3
97,high,4,low,low,low,high,74,2


PART 2

In [21]:
from pgmpy.estimators import PC
# estimatedDAG = PC(data.loc[:, data.columns != 'car name'])
#estimatedDAG.estimate().edges()
#https://pgmpy.org/_modules/pgmpy/estimators/PC.html
est = PC(dataStr)

skel, seperating_sets = est.build_skeleton(significance_level=0.01, show_progress= False )
    


c:\Users\Razi's good laptop\AppData\Local\Programs\Python\Python39\lib\site-packages\pgmpy\estimators\CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
c:\Users\Razi's good laptop\AppData\Local\Programs\Python\Python39\lib\site-packages\pgmpy\estimators\CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
c:\Users\Razi's good laptop\AppData\Local\Programs\Python\Python39\lib\site-packages\pgmpy\estimators\CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal t

In [22]:
print("Undirected edges: ", skel.edges())

pdag = est.skeleton_to_pdag(skel, seperating_sets)
print("PDAG edges:       ", pdag.edges())

model = pdag.to_dag()
print("DAG edges:        ", model.edges())

Undirected edges:  [('cylinders', 'displacement'), ('cylinders', 'acceleration'), ('displacement', 'origin')]
PDAG edges:        [('displacement', 'cylinders'), ('displacement', 'origin'), ('acceleration', 'cylinders'), ('origin', 'displacement')]
DAG edges:         [('displacement', 'cylinders'), ('acceleration', 'cylinders'), ('origin', 'displacement')]


PART 2B

BIC Score

In [23]:
from pgmpy.estimators import HillClimbSearch as hcs
from pgmpy.estimators import BDeuScore, K2Score, BicScore
climb = hcs(data)
bestBICHCmodel = climb.estimate(scoring_method=BicScore(data))
print(bestBICHCmodel.edges())

  0%|          | 7/1000000 [00:00<28:35:57,  9.71it/s]

[('mpg', 'horsepower'), ('cylinders', 'displacement'), ('cylinders', 'weight'), ('cylinders', 'mpg'), ('displacement', 'origin'), ('displacement', 'model year'), ('horsepower', 'acceleration')]


BDEUScore

In [24]:

bestBDSHCModel = climb.estimate(scoring_method=BDeuScore(data))
print(bestBDSHCModel.edges())

  0%|          | 17/1000000 [00:01<20:31:28, 13.53it/s]

[('mpg', 'weight'), ('mpg', 'model year'), ('cylinders', 'weight'), ('cylinders', 'mpg'), ('cylinders', 'horsepower'), ('cylinders', 'origin'), ('cylinders', 'acceleration'), ('cylinders', 'model year'), ('displacement', 'cylinders'), ('displacement', 'horsepower'), ('displacement', 'weight'), ('displacement', 'origin'), ('displacement', 'mpg'), ('horsepower', 'mpg'), ('horsepower', 'acceleration'), ('horsepower', 'weight'), ('weight', 'origin')]


K2 Score

In [25]:
bestK2HCModel = climb.estimate(scoring_method=K2Score(data))
print(bestK2HCModel.edges())
bestK2HCModel

  0%|          | 12/1000000 [00:01<25:01:58, 11.10it/s]

[('mpg', 'horsepower'), ('mpg', 'model year'), ('cylinders', 'displacement'), ('cylinders', 'weight'), ('cylinders', 'mpg'), ('cylinders', 'horsepower'), ('cylinders', 'acceleration'), ('cylinders', 'model year'), ('displacement', 'origin'), ('displacement', 'mpg'), ('horsepower', 'acceleration'), ('weight', 'mpg')]


PART 3

3 A

In [26]:
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.models import BayesianModel as bm
BICHModel1 = bm(bestBICHCmodel.edges())
mle = MaximumLikelihoodEstimator(BICHModel1, data)
mleCPDs = mle.get_parameters()

    
from pgmpy.factors.discrete.CPD import TabularCPD
#https://stackoverflow.com/questions/70625490/how-to-print-the-printing-full-cpd-from-pgmpy
#credit for printing untruncated tables to console...
def print_full(cpd):
    backup = TabularCPD._truncate_strtable
    TabularCPD._truncate_strtable = lambda self, x: x
    print(cpd)
    TabularCPD._truncate_strtable = backup
   



c:\Users\Razi's good laptop\AppData\Local\Programs\Python\Python39\lib\site-packages\pgmpy\models\BayesianModel.py:8: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  warnings.warn(


In [27]:
for cpd in mleCPDs:
    BICHModel1.add_cpds(cpd)
 
for cpd in mleCPDs:
    print_full(cpd)

for node in BICHModel1.nodes:
    print(f'{node}')
    print(BICHModel1.local_independencies(node),'\n')

+-----------+--------------+----------------------+--------------+--------------+
| cylinders | cylinders(3) | cylinders(4)         | cylinders(6) | cylinders(8) |
+-----------+--------------+----------------------+--------------+--------------+
| mpg(high) | 0.0          | 0.9583333333333334   | 0.25         | 0.0          |
+-----------+--------------+----------------------+--------------+--------------+
| mpg(low)  | 1.0          | 0.041666666666666664 | 0.75         | 1.0          |
+-----------+--------------+----------------------+--------------+--------------+
+------------------+---------------------+----------+
| mpg              | mpg(high)           | mpg(low) |
+------------------+---------------------+----------+
| horsepower(high) | 0.02040816326530612 | 0.9      |
+------------------+---------------------+----------+
| horsepower(low)  | 0.9795918367346939  | 0.1      |
+------------------+---------------------+----------+
+--------------+----------+
| cylinders(3) | 0.0

3 B

In [28]:
from pgmpy.estimators import ExpectationMaximization as EM
BICHModel2 = bm(bestBICHCmodel.edges())
em = EM(BICHModel2, data)
emCPDs = em.get_parameters()



c:\Users\Razi's good laptop\AppData\Local\Programs\Python\Python39\lib\site-packages\pgmpy\models\BayesianModel.py:8: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  warnings.warn(
  0%|          | 0/100 [00:00<?, ?it/s]c:\Users\Razi's good laptop\AppData\Local\Programs\Python\Python39\lib\site-packages\pgmpy\estimators\base.py:156: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data.groupby([variable] + parents).sum()["_weight"].unstack(parents)
c:\Users\Razi's good laptop\AppData\Local\Programs\Python\Python39\lib\site-packages\pgmpy\estimators\base.py:156: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Eit

In [29]:
for cpd in emCPDs:
    BICHModel2.add_cpds(cpd)
for cpd in emCPDs:
    print_full(cpd)

for node in BICHModel2.nodes:
    print(f'{node}')
    print(BICHModel2.local_independencies(node),'\n')

+-----------+--------------+----------------------+--------------+--------------+
| cylinders | cylinders(3) | cylinders(4)         | cylinders(6) | cylinders(8) |
+-----------+--------------+----------------------+--------------+--------------+
| mpg(high) | 0.0          | 0.9583333333333334   | 0.25         | 0.0          |
+-----------+--------------+----------------------+--------------+--------------+
| mpg(low)  | 1.0          | 0.041666666666666664 | 0.75         | 1.0          |
+-----------+--------------+----------------------+--------------+--------------+
+------------------+---------------------+----------+
| mpg              | mpg(high)           | mpg(low) |
+------------------+---------------------+----------+
| horsepower(high) | 0.02040816326530612 | 0.9      |
+------------------+---------------------+----------+
| horsepower(low)  | 0.9795918367346939  | 0.1      |
+------------------+---------------------+----------+
+--------------+----------+
| cylinders(3) | 0.0